In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import skewnorm 
import sqlite3
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
#import data from csv file
df_csv1 = pd.read_csv("online_retail.csv", encoding="unicode_escape") 
df_csv2 = pd.read_csv("online_retail_II.csv",  encoding="unicode_escape")
df_csv = pd.concat([df_csv1, df_csv2], axis=0)
df_csv.columns = df_csv.columns.str.lower().str.strip()
df_csv["invoicedate"] = pd.to_datetime(df_csv["invoicedate"], format="%d/%m/%Y %H:%M")

In [3]:
df_csv = df_csv[df_csv["description"].notna()]
df_csv = df_csv[df_csv["quantity"] > 0]
df_csv = df_csv[df_csv["price"]> 0]
df_csv = df_csv[df_csv["country"] == "United Kingdom"]

In [4]:
df_csv["description"] = df_csv["description"].str.strip()
df_csv["invoice"] = df_csv["invoice"].astype("str")
df_csv["stockcode"] = df_csv["stockcode"].astype("str")
df_csv

,invoice,stockcode,description,quantity,invoicedate,price,customer id,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom


In [5]:
basket_plus = (df_csv[df_csv['country'] =="United Kingdom"].groupby(['invoice', 'description'])['quantity']
               .sum().unstack().reset_index().fillna(0).set_index('invoice'))

In [6]:
basket_plus

description,*Boombox Ipod Classic,*USB Office Glitter Lamp,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,11 PC CERAMIC TEA SET POLKADOT,12 ASS ZINC CHRISTMAS DECORATIONS,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
invoice,,,,,,,,,,,,,,,,,,,,,
489434,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
489435,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
489436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
489437,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
489438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581584,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,12.0,0.0,0.0,0.0,24.0,0.0,0.0
581586,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
baskets = (df_csv.groupby(['invoice','description'])['stockcode'].sum().unstack().reset_index().fillna(0).set_index('invoice'))

In [8]:
baskets

description,*Boombox Ipod Classic,*USB Office Glitter Lamp,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,11 PC CERAMIC TEA SET POLKADOT,12 ASS ZINC CHRISTMAS DECORATIONS,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
invoice,,,,,,,,,,,,,,,,,,,,,
489434,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489435,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489436,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489437,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489438,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581584,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581585,0,0,0,0,0,0,0,0,0,0,...,0,0,0,23145,0,0,0,84832,0,0
581586,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Converting table to a list containing transactions with stock codes 

In [9]:
transaction_array_full = baskets.values.tolist()

Removing items that were't purchased

In [10]:
transaction_array = []
for i in transaction_array_full:
    a = [j for j in i if j!= 0]
    transaction_array.append(a)

Creating a list of every single item purchased with repetition to count how many times an item was purchased overall

In [11]:
flat_trans_array = [i for j in transaction_array for i in j]
len(flat_trans_array)

914178

Creating a dictionary for each item containing its total purchases and support (appearances/transactions)

In [12]:
count_dict = {}
for i in flat_trans_array:
    if i not in count_dict.keys():
        item_count = flat_trans_array.count(i)
        item_support = item_count/len(transaction_array)
        item_data = (item_count,item_support)
        count_dict[i] = item_data

In [13]:
count_dict

{'85048': (452, 0.012371359754762426),
 '21523': (1016, 0.027808189183271294),
 '79323P': (253, 0.006924677030873659),
 '22064': (610, 0.016695861615940444),
 '22041': (519, 0.014205167506021459),
 '21871': (678, 0.01855703963214364),
 '21232': (2157, 0.05903766148456317),
 '79323W': (339, 0.00927851981607182),
 '22350': (295, 0.008074228158528575),
 '22349': (338, 0.009251149551127654),
 '22195': (707, 0.019350777315524415),
 '22353': (401, 0.010975476242610029),
 '82582': (746, 0.020418217648346837),
 '84879': (2528, 0.06919202977884827),
 '21756': (530, 0.01450624042040727),
 '84596L': (240, 0.006568863586599518),
 '22194': (147, 0.004023428946792205),
 '22142': (442, 0.01209765710532078),
 '21333': (203, 0.005556163783665426),
 '48173C': (468, 0.012809283993869061),
 '22109': (592, 0.01620319684694548),
 '22295': (636, 0.017407488504488724),
 '22296': (749, 0.020500328443179328),
 '21754': (1909, 0.05224983577841034),
 '21755': (1585, 0.043381869936500984),
 '22119': (302, 0.008265

Creating a dictionary of all items with a support > min support as per apriori algorithm pruning principles

In [15]:
support_threshold = 0.025
high_support = {}
for i,j in count_dict.items():
    if j[1] > support_threshold:
        high_support[i] = j[1]

In [18]:
high_support

{'21523': 0.027808189183271294,
 '21232': 0.05903766148456317,
 '84879': 0.06919202977884827,
 '21754': 0.05224983577841034,
 '21755': 0.043381869936500984,
 '21181': 0.04551675060214583,
 '22111': 0.04056273264725203,
 '22112': 0.04236917013356689,
 '84970S': 0.03601926866652069,
 '20971': 0.0281913728924896,
 '20711': 0.027123932559667177,
 '22138': 0.04417560761988176,
 '84029E': 0.026877600175169696,
 '21790': 0.04669367199474491,
 '85123A': 0.13348478213269105,
 '82580': 0.03194109918984016,
 '85150': 0.028410335012042917,
 '21929': 0.04048062185241953,
 '22219': 0.02947777534486534,
 '22356': 0.03744252244361725,
 '22114': 0.03881103569082549,
 '85099B': 0.0981223998248303,
 '21928': 0.039084738340267135,
 '20728': 0.05019706590759799,
 '20725': 0.07244909130720385,
 '20726': 0.04376505364571929,
 '22086': 0.04934858769432888,
 '22083': 0.03486971753886577,
 '21733': 0.043217648346835995,
 '21485': 0.032515874753667615,
 '20724': 0.04658419093496825,
 '22178': 0.04628311802058244

Support = (Transactions Containing X and Y)/Total Transactions
Confidence = (Transactions Containing X and Y)/Transactions containing X

Using items with high support, creating a dictionary of each item paired with each other item and calculating ~~support~~ *confidence* for that particular itemset

item_sets = {}
for item1 in high_support.keys():
    supp_dict = {}
    total_apperances = count_dict[item1][0]
    for item2 in high_support.keys():
        if item2 != item1:
            pair_count = 0
            for transaction in transaction_array:
                if item2 in transaction and item1 in transaction:
                    pair_count += 1
            supp_dict[item2] = pair_count/total_apperances
    item_sets[item1] = supp_dict

This time calculating **support** for an itemset

In [19]:
item_set_support = {}
for item1 in high_support.keys():
    supp_dict = {}
    total_apperances = count_dict[item1][0]
    for item2 in high_support.keys():
        if item2 != item1:
            pair_count = 0
            for transaction in transaction_array:
                if item2 in transaction and item1 in transaction:
                    pair_count += 1
            supp_dict[item2] = pair_count/len(transaction_array)
    item_set_support[item1] = supp_dict


In [20]:
item_set_support

{'21523': {'21232': 0.004680315305452157,
  '84879': 0.002955988613969783,
  '21754': 0.004954017954893804,
  '21755': 0.003968688416903876,
  '21181': 0.002983358878913948,
  '22111': 0.0026275454346398074,
  '22112': 0.003038099408802277,
  '84970S': 0.0030107291438581127,
  '20971': 0.0016148456317057148,
  '20711': 0.002326472520253996,
  '22138': 0.0018338077512590322,
  '84029E': 0.0020527698708123496,
  '21790': 0.003530764177797241,
  '85123A': 0.007827895774031092,
  '82580': 0.0029286183490256188,
  '85150': 0.0021075104007006787,
  '21929': 0.0033391723231880883,
  '22219': 0.002901248084081454,
  '22356': 0.0034212831180205822,
  '22114': 0.0032296912634114296,
  '85099B': 0.006158309612437049,
  '21928': 0.0021348806656448435,
  '20728': 0.0025180643748631487,
  '20725': 0.004078169476680534,
  '20726': 0.003476023647908912,
  '22086': 0.002901248084081454,
  '22083': 0.00281913728924896,
  '21733': 0.0031475804685789357,
  '21485': 0.0022169914604773374,
  '20724': 0.0039

Removing itemn set pairs that fall below minimum support

In [21]:
frequent_item_sets = {}
for i,j in item_set_support.items():
    common_dict = {}
    for k,l in j.items():
        if l > support_threshold:
            common_dict[k] = l
    if not len(common_dict.keys()) == 0:
        frequent_item_sets[i] = common_dict

frequent_item_sets

{'21232': {'21231': 0.026412305671118897},
 '85123A': {'21733': 0.02925881322531202},
 '85099B': {'20725': 0.025837530107291437,
  '21931': 0.03336435296693672,
  '85099C': 0.028109262097657104,
  '85099F': 0.029641996934530325,
  '22386': 0.03484234727392161,
  '22411': 0.029368294285088678},
 '20725': {'85099B': 0.025837530107291437,
  '20727': 0.029121961900591197,
  '22383': 0.027643967593606305,
  '22384': 0.02701445149989052},
 '21733': {'85123A': 0.02925881322531202},
 '84991': {'21212': 0.027260783884388},
 '21212': {'84991': 0.027260783884388},
 '82494L': {'82482': 0.027041821764834682},
 '21231': {'21232': 0.026412305671118897},
 '21931': {'85099B': 0.03336435296693672},
 '20727': {'20725': 0.029121961900591197},
 '85099C': {'85099B': 0.028109262097657104},
 '82482': {'82494L': 0.027041821764834682},
 '85099F': {'85099B': 0.029641996934530325},
 '22383': {'20725': 0.027643967593606305},
 '22384': {'20725': 0.02701445149989052},
 '22386': {'85099B': 0.03484234727392161},
 '224

In [45]:
lift_dict = {}
for item1,item_sets in frequent_item_sets.items():
    for item2,item_support in item_sets.items():
        lift_dict[(item1,item2)] = (item_support/high_support[item1]*high_support[item2])*100

lift_dict
    

{('21232', '21231'): 1.6310241610537957,
 ('85123A', '21733'): 0.9472968235137929,
 ('85099B', '20725'): 1.907725026331951,
 ('85099B', '21931'): 2.013029162272918,
 ('85099B', '85099C'): 1.3862531489165346,
 ('85099B', '85099F'): 1.4105787104688632,
 ('85099B', '22386'): 1.9894640131023025,
 ('85099B', '22411'): 1.6646129424630463,
 ('20725', '85099B'): 3.4993405906550734,
 ('20725', '20727'): 2.350000401196177,
 ('20725', '22383'): 2.2328221652863722,
 ('20725', '22384'): 1.9135661716016141,
 ('21733', '85123A'): 9.037063464208153,
 ('84991', '21212'): 3.836869252920728,
 ('21212', '84991'): 1.9368664632645551,
 ('82494L', '82482'): 2.779544630582188,
 ('21231', '21232'): 4.277127877823082,
 ('21931', '85099B'): 5.529875422397972,
 ('20727', '20725'): 3.6088873197970455,
 ('85099C', '85099B'): 5.69975704864823,
 ('82482', '82494L'): 2.6308630425118276,
 ('85099F', '85099B'): 6.228989390990106,
 ('22383', '20725'): 3.422524893371183,
 ('22384', '20725'): 3.813720166411211,
 ('22386', 

## Adjustment 

In [22]:
count_threshold = 1000
high_count = {}
for i,j in count_dict.items():
    if j[0] > count_threshold:
        high_count[i] = j[0]

In [23]:
high_count

{'21523': 1016,
 '21232': 2157,
 '84879': 2528,
 '21754': 1909,
 '21755': 1585,
 '21181': 1663,
 '22111': 1482,
 '22112': 1548,
 '84970S': 1316,
 '20971': 1030,
 '22138': 1614,
 '21790': 1706,
 '85123A': 4877,
 '82580': 1167,
 '85150': 1038,
 '21929': 1479,
 '22219': 1077,
 '22356': 1368,
 '22114': 1418,
 '85099B': 3585,
 '21928': 1428,
 '20728': 1834,
 '20725': 2647,
 '20726': 1599,
 '22086': 1803,
 '22083': 1274,
 '21733': 1579,
 '21485': 1188,
 '20724': 1702,
 '22178': 1691,
 '84991': 1821,
 '21977': 1721,
 '21731': 1102,
 '22139': 1787,
 '21212': 2563,
 '82483': 1045,
 '82486': 1172,
 '82494L': 1830,
 '21930': 1239,
 '20972': 1284,
 '21231': 1332,
 '47591D': 1003,
 '21791': 1134,
 '20719': 1214,
 '21931': 2163,
 '84946': 1652,
 '85152': 1376,
 '22077': 1490,
 '21166': 1295,
 '48194': 1262,
 '21175': 1547,
 '82600': 1126,
 '22197': 2056,
 '21174': 1075,
 '20685': 1513,
 '21034': 1649,
 '84836': 1271,
 '22355': 1531,
 '22189': 1221,
 '20713': 1198,
 '20712': 1170,
 '20727': 2136,
 '2

In [32]:
item_set_support_count = {}
for item1 in high_count.keys():
    supp_count_dict = {}
    for item2 in high_count.keys():
        if item2 != item1:
            pair_count = 0
            for transaction in transaction_array:
                if item2 in transaction and item1 in transaction:
                    pair_count += 1
            supp_count_dict[item2] = pair_count
    item_set_support_count[item1] = supp_count_dict

In [33]:
item_set_support_count

{'21523': {'21232': 171,
  '84879': 108,
  '21754': 181,
  '21755': 145,
  '21181': 109,
  '22111': 96,
  '22112': 111,
  '84970S': 110,
  '20971': 59,
  '22138': 67,
  '21790': 129,
  '85123A': 286,
  '82580': 107,
  '85150': 77,
  '21929': 122,
  '22219': 106,
  '22356': 125,
  '22114': 118,
  '85099B': 225,
  '21928': 78,
  '20728': 92,
  '20725': 149,
  '20726': 127,
  '22086': 106,
  '22083': 103,
  '21733': 115,
  '21485': 81,
  '20724': 145,
  '22178': 108,
  '84991': 120,
  '21977': 122,
  '21731': 108,
  '22139': 93,
  '21212': 177,
  '82483': 104,
  '82486': 98,
  '82494L': 136,
  '21930': 102,
  '20972': 64,
  '21231': 95,
  '47591D': 48,
  '21791': 93,
  '20719': 126,
  '21931': 161,
  '84946': 133,
  '85152': 86,
  '22077': 126,
  '21166': 82,
  '48194': 202,
  '21175': 128,
  '82600': 77,
  '22197': 128,
  '21174': 94,
  '20685': 248,
  '21034': 32,
  '84836': 78,
  '22355': 139,
  '22189': 100,
  '20713': 95,
  '20712': 101,
  '20727': 121,
  '21080': 113,
  '20914': 92,

In [35]:
frequent_item_sets_count = {}
for i,j in item_set_support_count.items():
    common_dict = {}
    for k,l in j.items():
        if l > count_threshold:
            common_dict[k] = l
    if not len(common_dict.keys()) == 0:
        frequent_item_sets_count[i] = common_dict

frequent_item_sets_count

{'85123A': {'21733': 1069},
 '85099B': {'21931': 1219,
  '85099C': 1027,
  '85099F': 1083,
  '22386': 1273,
  '22411': 1073},
 '20725': {'20727': 1064, '22383': 1010},
 '21733': {'85123A': 1069},
 '21931': {'85099B': 1219},
 '20727': {'20725': 1064},
 '85099C': {'85099B': 1027},
 '85099F': {'85099B': 1083},
 '22383': {'20725': 1010},
 '22386': {'85099B': 1273},
 '22411': {'85099B': 1073}}

In [42]:
support_table = pd.DataFrame.from_dict(frequent_item_sets_count)
support_table = support_table.fillna(0)
support_table

,85123A,85099B,20725,21733,21931,20727,85099C,85099F,22383,22386,22411
21733,1069.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21931,0.0,1219.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
85099C,0.0,1027.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
85099F,0.0,1083.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22386,0.0,1273.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22411,0.0,1073.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20727,0.0,0.0,1064.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22383,0.0,0.0,1010.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
85123A,0.0,0.0,0.0,1069.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
85099B,0.0,0.0,0.0,0.0,1219.0,0.0,1027.0,1083.0,0.0,1273.0,1073.0
